# Exercise 9: Retrieval Score Analysis

**Goal:** Understand what similarity scores actually mean — when can you trust them, when are they ambiguous, and what threshold separates signal from noise?

| Concept | Meaning |
|---|---|
| **Clear winner** | Large gap between score #1 and #2 — retriever is confident |
| **Tight cluster** | All scores similar — retriever is uncertain which chunk is best |
| **Score threshold** | Minimum score below which a chunk is considered irrelevant |

**For each of 10 queries this notebook will:**
1. Retrieve top-10 chunks and record all scores
2. Compute gap, spread, and cluster statistics
3. Plot score distributions
4. Run RAG with and without a score threshold and compare answers
5. Save everything to CSV

---
## Setup

In [ ]:
try:
    ip = get_ipython()
    ip.run_line_magic('pip', 'install -q torch transformers sentence-transformers faiss-cpu pymupdf accelerate ipyfilechooser pandas matplotlib')
except NameError:
    import subprocess, sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q',
        'torch', 'transformers', 'sentence-transformers',
        'faiss-cpu', 'pymupdf', 'accelerate', 'ipyfilechooser', 'pandas', 'matplotlib'])

In [ ]:
import os, time
import torch
import faiss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from pathlib import Path
from dataclasses import dataclass
from typing import List, Tuple
import fitz  # PyMuPDF

os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

def get_device():
    if torch.cuda.is_available():
        name = torch.cuda.get_device_name(0)
        mem  = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f'✔ CUDA GPU: {name} ({mem:.1f} GB)')
        return 'cuda', torch.float16
    elif torch.backends.mps.is_available():
        print('✔ Apple Silicon MPS')
        return 'mps', torch.float32
    else:
        print('⚠ CPU only')
        return 'cpu', torch.float32

try:
    import google.colab
    ENVIRONMENT = 'colab'
except ImportError:
    ENVIRONMENT = 'local'

DEVICE, DTYPE = get_device()
print(f'Environment: {ENVIRONMENT.upper()} | Device: {DEVICE}')

---
## Load Documents

In [ ]:
# =============================================================================
# CELL 1 — SELECT DOCUMENT SOURCE  (DO NOT CHANGE)
# =============================================================================
USE_GOOGLE_DRIVE = True
DOC_FOLDER = 'documents'
folder_chooser = None

if ENVIRONMENT == 'colab':
    if USE_GOOGLE_DRIVE:
        from google.colab import drive
        print('Mounting Google Drive...')
        drive.mount('/content/drive')
        print('✔ Google Drive mounted\n')
        try:
            from ipyfilechooser import FileChooser
            folder_chooser = FileChooser(
                path='/content/drive/MyDrive',
                title='Select your documents folder in Google Drive',
                show_only_dirs=True, select_default=True)
            print('📁 Select your documents folder below, then run Cell 2:')
            display(folder_chooser)
        except ImportError:
            DOC_FOLDER = '/content/drive/MyDrive/your_documents_folder'
            print(f"Edit DOC_FOLDER: '{DOC_FOLDER}'")
    else:
        from google.colab import files as colab_files
        os.makedirs(DOC_FOLDER, exist_ok=True)
        uploaded = colab_files.upload()
        for fn in uploaded:
            os.rename(fn, f'{DOC_FOLDER}/{fn}')
else:
    try:
        from ipyfilechooser import FileChooser
        folder_chooser = FileChooser(path=str(Path.home()),
            title='Select documents folder', show_only_dirs=True, select_default=True)
        display(folder_chooser)
    except ImportError:
        print(f'Using default folder: {DOC_FOLDER}')

In [ ]:
# =============================================================================
# CELL 2 — CONFIRM SELECTION  (DO NOT CHANGE)
# =============================================================================
if folder_chooser is not None and folder_chooser.selected_path:
    DOC_FOLDER = folder_chooser.selected_path
    print(f'✔ Using: {DOC_FOLDER}')
elif folder_chooser is not None:
    print('⚠ No folder selected — go back, select one, then re-run this cell.')
else:
    print(f'✔ Using: {DOC_FOLDER}')

In [ ]:
def load_documents(folder: str) -> List[Tuple[str, str]]:
    docs = []
    p = Path(folder)
    if not p.exists():
        print(f'⚠ Folder not found: {folder}')
        return docs
    for path in sorted(p.iterdir()):
        if path.suffix.lower() == '.pdf':
            doc  = fitz.open(str(path))
            text = ''.join(f'[Page {i+1}]\n{page.get_text()}\n' for i, page in enumerate(doc))
            docs.append((path.name, text))
        elif path.suffix.lower() in ('.txt', '.md'):
            docs.append((path.name, path.read_text(encoding='utf-8', errors='replace')))
    print(f'✔ Loaded {len(docs)} documents')
    for name, content in docs:
        print(f'   {name}: {len(content):,} chars')
    return docs

documents = load_documents(DOC_FOLDER)

---
## Build the RAG Pipeline
Fixed chunk size (512) and overlap (128) — same as Exercise 1 defaults.
Score analysis is the variable here, not chunk size.

In [ ]:
@dataclass
class Chunk:
    text: str
    source_file: str
    chunk_index: int
    start_char: int
    end_char: int


def chunk_text(text: str, source_file: str,
               chunk_size: int, chunk_overlap: int) -> list:
    chunks, start, idx = [], 0, 0
    while start < len(text):
        end = start + chunk_size
        if end < len(text):
            pb = text.rfind('\n\n', start + chunk_size // 2, end)
            if pb != -1:
                end = pb + 2
            else:
                sb = text.rfind('. ', start + chunk_size // 2, end)
                if sb != -1:
                    end = sb + 2
        s = text[start:end].strip()
        if s:
            chunks.append(Chunk(s, source_file, idx, start, end))
            idx += 1
        prev  = start
        start = end - chunk_overlap
        if start <= prev:
            start = end
    return chunks


def rebuild_pipeline(chunk_size: int = 512, chunk_overlap: int = 128):
    """Re-chunk documents, re-embed, rebuild FAISS index. Updates globals."""
    global all_chunks, index
    all_chunks = []
    for filename, content in documents:
        all_chunks.extend(
            chunk_text(content, filename,
                       chunk_size=chunk_size,
                       chunk_overlap=chunk_overlap)
        )
    chunk_embeddings = embed_model.encode(
        [c.text for c in all_chunks],
        show_progress_bar=True
    ).astype('float32')
    faiss.normalize_L2(chunk_embeddings)
    index = faiss.IndexFlatIP(EMBEDDING_DIM)
    index.add(chunk_embeddings)
    print(f'Rebuilt: {len(all_chunks)} chunks, '
          f'chunk_size={chunk_size}, chunk_overlap={chunk_overlap}')


all_chunks = []
index      = None
print('✔ Chunk + pipeline functions defined')

In [ ]:
from sentence_transformers import SentenceTransformer

EMBED_MODEL_NAME = 'all-MiniLM-L6-v2'
EMBEDDING_DIM    = 384

print(f'Loading embedding model: {EMBED_MODEL_NAME} ...')
embed_model = SentenceTransformer(EMBED_MODEL_NAME, device=DEVICE)
print('✔ Embedding model ready')

# Build index with Exercise 1 defaults
CHUNK_SIZE    = 512
CHUNK_OVERLAP = 128
rebuild_pipeline(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

LLM_MODEL = 'Qwen/Qwen2.5-1.5B-Instruct'
print(f'Loading LLM: {LLM_MODEL} ...')
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)

if DEVICE == 'cuda':
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL, device_map='auto', dtype=DTYPE, trust_remote_code=True)
elif DEVICE == 'mps':
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL, dtype=DTYPE, trust_remote_code=True).to(DEVICE)
else:
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL, dtype=DTYPE, trust_remote_code=True)

print(f'✔ LLM loaded on {DEVICE}')

---
## Define 10 Queries
Use a mix of types to see how score distributions differ by question difficulty.

In [ ]:
# =============================================================================
# CONFIGURE YOUR 10 QUERIES HERE
# =============================================================================
QUERIES = [
    {'id': 'Q01', 'type': 'narrow_factual',
     'text': 'What is the correct spark plug gap for a Model T Ford?'},

    {'id': 'Q02', 'type': 'procedural',
     'text': 'How do I fix a slipping transmission band?'},

    {'id': 'Q03', 'type': 'procedural',
     'text': 'How do I adjust the carburetor on a Model T?'},

    {'id': 'Q04', 'type': 'broad_conceptual',
     'text': 'What oil should I use in a Model T engine?'},

    {'id': 'Q05', 'type': 'multi_section',
     'text': 'What are all the steps to prepare a Model T for winter driving?'},

    {'id': 'Q06', 'type': 'narrow_factual',
     'text': 'How do I start a Model T Ford?'},

    {'id': 'Q07', 'type': 'procedural',
     'text': 'How do I check and fill the radiator on a Model T?'},

    {'id': 'Q08', 'type': 'narrow_factual',
     'text': 'What does the hand throttle lever do on a Model T?'},

    {'id': 'Q09', 'type': 'off_topic',
     'text': 'What is the latest iPhone model released by Apple?'},

    {'id': 'Q10', 'type': 'ambiguous',
     'text': 'How does the Ford engine work?'},
]

TOP_K = 10   # always retrieve top-10 for score analysis

print(f'✔ {len(QUERIES)} queries defined (TOP_K={TOP_K})')
for q in QUERIES:
    print(f"   [{q['id']}] ({q['type']:<16}) {q['text']}")

---
## Retrieve & Record All Scores

In [ ]:
PROMPT_TEMPLATE = (
    'You are a helpful assistant. Answer the question using ONLY the context below.\n'
    'If the context does not contain enough information, say so.\n\n'
    'CONTEXT:\n{context}\n\n'
    'QUESTION: {question}\n\n'
    'ANSWER:'
)


def generate_response(prompt: str, max_new_tokens: int = 256) -> str:
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(
        out[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    ).strip()


def retrieve_with_scores(question: str, top_k: int = 10) -> list:
    """Return list of (chunk, score) for top_k results."""
    q_emb = embed_model.encode([question]).astype('float32')
    faiss.normalize_L2(q_emb)
    scores, indices = index.search(q_emb, top_k)
    return [
        (all_chunks[i], float(s))
        for s, i in zip(scores[0], indices[0]) if i != -1
    ]


def rag_with_threshold(question: str, retrieved: list,
                        threshold: float = None) -> Tuple[str, int]:
    """
    Generate an answer using only chunks that pass the score threshold.
    Returns (answer, n_chunks_used).
    """
    if threshold is not None:
        kept = [(c, s) for c, s in retrieved if s >= threshold]
    else:
        kept = retrieved

    if not kept:
        return 'No chunks passed the score threshold.', 0

    context = '\n\n---\n\n'.join(
        f'[Source: {c.source_file} | Score: {s:.3f}]\n{c.text}'
        for c, s in kept
    )
    prompt = PROMPT_TEMPLATE.format(context=context, question=question)
    return generate_response(prompt), len(kept)


print('✔ Retrieval and generation helpers defined')

In [ ]:
# =============================================================================
# SCORE THRESHOLD TO TEST
# =============================================================================
THRESHOLD = 0.4   # ← adjust after inspecting the score distributions below

# ── Run all 10 queries ────────────────────────────────────────────────────────
score_rows   = []   # one row per (query × rank) for detailed score CSV
summary_rows = []   # one row per query for summary CSV

for q in QUERIES:
    print(f"\n[{q['id']}] ({q['type']}) {q['text']}")

    retrieved = retrieve_with_scores(q['text'], top_k=TOP_K)
    scores    = [s for _, s in retrieved]

    # ── Score statistics ──────────────────────────────────────────────────────
    top1_score    = scores[0] if scores else 0.0
    top2_score    = scores[1] if len(scores) > 1 else 0.0
    winner_gap    = top1_score - top2_score          # large = clear winner
    score_spread  = scores[0] - scores[-1] if scores else 0.0
    score_std     = float(np.std(scores))            # low = tight cluster
    above_thresh  = sum(1 for s in scores if s >= THRESHOLD)

    print(f'  top1={top1_score:.4f}  top2={top2_score:.4f}  '
          f'gap={winner_gap:.4f}  std={score_std:.4f}  '
          f'above_threshold({THRESHOLD})={above_thresh}/{TOP_K}')

    # ── Answers: no threshold vs. with threshold ──────────────────────────────
    answer_no_thresh, n_no_thresh   = rag_with_threshold(q['text'], retrieved, threshold=None)
    answer_with_thresh, n_with_thresh = rag_with_threshold(q['text'], retrieved, threshold=THRESHOLD)

    print(f'  No threshold  ({n_no_thresh} chunks): {answer_no_thresh[:120]}...')
    print(f'  Threshold≥{THRESHOLD} ({n_with_thresh} chunks): {answer_with_thresh[:120]}...')

    # ── Per-rank rows for detailed CSV ────────────────────────────────────────
    for rank, (chunk, score) in enumerate(retrieved, start=1):
        score_rows.append({
            'query_id'      : q['id'],
            'query_type'    : q['type'],
            'question'      : q['text'],
            'rank'          : rank,
            'score'         : round(score, 4),
            'above_threshold': score >= THRESHOLD,
            'source'        : chunk.source_file,
            'char_len'      : len(chunk.text),
            'chunk_preview' : chunk.text[:150].replace('\n', ' '),
        })

    # ── Summary row ───────────────────────────────────────────────────────────
    summary_rows.append({
        'query_id'           : q['id'],
        'query_type'         : q['type'],
        'question'           : q['text'],
        'top1_score'         : round(top1_score, 4),
        'top2_score'         : round(top2_score, 4),
        'winner_gap'         : round(winner_gap, 4),
        'score_spread'       : round(score_spread, 4),
        'score_std'          : round(score_std, 4),
        'n_above_threshold'  : above_thresh,
        'threshold'          : THRESHOLD,
        'answer_no_thresh'   : answer_no_thresh,
        'answer_with_thresh' : answer_with_thresh,
        'chunks_no_thresh'   : n_no_thresh,
        'chunks_with_thresh' : n_with_thresh,
    })

# Save immediately
df_scores  = pd.DataFrame(score_rows)
df_summary = pd.DataFrame(summary_rows)
df_scores .to_csv('exercise9_score_details.csv',  index=False)
df_summary.to_csv('exercise9_score_summary.csv', index=False)
print('\n✔ Saved: exercise9_score_details.csv  exercise9_score_summary.csv')

---
## Visualise Score Distributions

In [ ]:
# ── Plot 1: Score profile per query (rank 1–10 on x-axis) ────────────────────
fig, axes = plt.subplots(2, 5, figsize=(18, 7), sharey=True)
axes = axes.flatten()

for ax, q in zip(axes, QUERIES):
    sub    = df_scores[df_scores['query_id'] == q['id']].sort_values('rank')
    ranks  = sub['rank'].values
    scores = sub['score'].values
    colors = ['#e15759' if s >= THRESHOLD else '#aec7e8' for s in scores]

    ax.bar(ranks, scores, color=colors)
    ax.axhline(THRESHOLD, color='black', linestyle='--', linewidth=0.8,
               label=f'threshold={THRESHOLD}')
    ax.set_title(f"{q['id']} ({q['type'][:10]})", fontsize=8)
    ax.set_xlabel('Rank', fontsize=7)
    ax.set_ylabel('Score', fontsize=7)
    ax.tick_params(labelsize=7)
    ax.set_xticks(ranks)
    ax.set_ylim(0, max(df_scores['score'].max() * 1.1, THRESHOLD * 1.2))

    # Annotate top-1 score
    ax.text(1, scores[0] + 0.01, f'{scores[0]:.3f}',
            ha='center', va='bottom', fontsize=7, fontweight='bold')

plt.suptitle(
    f'Retrieval Score Profiles (top-{TOP_K} chunks per query)\n'
    f'Red = above threshold ({THRESHOLD}), Blue = below',
    fontsize=11
)
plt.tight_layout(rect=[0, 0, 1, 0.93])
plt.savefig('exercise9_score_profiles.png', dpi=150, bbox_inches='tight')
plt.show()
print('✔ Saved: exercise9_score_profiles.png')

In [ ]:
# ── Plot 2: Winner gap vs. score spread — bubble chart ───────────────────────
fig, ax = plt.subplots(figsize=(9, 5))

cmap   = plt.get_cmap('tab10')
types  = df_summary['query_type'].unique()
colour = {t: cmap(i) for i, t in enumerate(types)}

for _, row in df_summary.iterrows():
    ax.scatter(
        row['winner_gap'], row['score_spread'],
        s=row['top1_score'] * 800,
        color=colour[row['query_type']],
        alpha=0.7, edgecolors='black', linewidths=0.5
    )
    ax.annotate(
        row['query_id'],
        (row['winner_gap'], row['score_spread']),
        textcoords='offset points', xytext=(6, 4), fontsize=8
    )

# Legend for query type
handles = [plt.Line2D([0], [0], marker='o', color='w',
           markerfacecolor=c, markersize=9, label=t)
           for t, c in colour.items()]
ax.legend(handles=handles, title='Query type', fontsize=8, loc='upper left')

ax.set_xlabel('Winner Gap  (score[1] − score[2])', fontsize=10)
ax.set_ylabel('Score Spread  (score[1] − score[10])', fontsize=10)
ax.set_title('Retrieval Confidence: Gap vs. Spread\n'
             '(bubble size = top-1 score)', fontsize=11)
ax.axvline(0, color='grey', linewidth=0.5)
ax.axhline(0, color='grey', linewidth=0.5)
plt.tight_layout()
plt.savefig('exercise9_gap_vs_spread.png', dpi=150, bbox_inches='tight')
plt.show()
print('✔ Saved: exercise9_gap_vs_spread.png')

In [ ]:
# ── Plot 3: Summary table — top scores + gap + std + chunks above threshold ──
print('=== SCORE SUMMARY TABLE ===')
pd.set_option('display.max_colwidth', 55)
display(df_summary[[
    'query_id', 'query_type', 'top1_score', 'top2_score',
    'winner_gap', 'score_std', 'n_above_threshold'
]].sort_values('winner_gap', ascending=False))

print(f'\nThreshold used: {THRESHOLD}')
print(f'Queries with clear winner (gap > 0.05): '
      f"{(df_summary['winner_gap'] > 0.05).sum()}")
print(f'Queries with tight cluster  (std < 0.02): '
      f"{(df_summary['score_std'] < 0.02).sum()}")
print(f'Queries with 0 chunks above threshold: '
      f"{(df_summary['n_above_threshold'] == 0).sum()}")

In [ ]:
# ── Plot 4: Threshold sensitivity — how many chunks survive at different thresholds? ──
thresholds = np.arange(0.1, 0.8, 0.05)
threshold_data = []

for t in thresholds:
    n_pass = (df_scores['score'] >= t).sum()
    q_pass = df_scores[df_scores['score'] >= t]['query_id'].nunique()
    threshold_data.append({'threshold': round(t, 2),
                            'chunks_above': n_pass,
                            'queries_with_any_chunk': q_pass})

df_thresh = pd.DataFrame(threshold_data)

fig, ax1 = plt.subplots(figsize=(9, 4))
ax2 = ax1.twinx()

ax1.bar(df_thresh['threshold'], df_thresh['chunks_above'],
        width=0.04, alpha=0.6, color='steelblue', label='Chunks above threshold')
ax2.plot(df_thresh['threshold'], df_thresh['queries_with_any_chunk'],
         color='crimson', marker='o', linewidth=2, label='Queries with ≥1 chunk')

ax1.axvline(THRESHOLD, color='black', linestyle='--', linewidth=1,
            label=f'Current threshold ({THRESHOLD})')

ax1.set_xlabel('Score Threshold', fontsize=10)
ax1.set_ylabel('Chunks Surviving', fontsize=10, color='steelblue')
ax2.set_ylabel('Queries with ≥1 Chunk', fontsize=10, color='crimson')
ax2.set_ylim(0, len(QUERIES) + 1)
ax2.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, fontsize=8, loc='upper right')

plt.title('Threshold Sensitivity: How Many Chunks / Queries Survive?', fontsize=11)
plt.tight_layout()
plt.savefig('exercise9_threshold_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()
print('✔ Saved: exercise9_threshold_sensitivity.png')

---
## Answer Comparison: No Threshold vs. Threshold

In [ ]:
pd.set_option('display.max_colwidth', 300)

for _, row in df_summary.iterrows():
    print(f"\n{'='*65}")
    print(f"[{row['query_id']}] {row['question']}")
    print(f"  top1={row['top1_score']}  gap={row['winner_gap']}  "
          f"chunks_above_threshold={row['n_above_threshold']}")
    print(f"\n  NO THRESHOLD ({row['chunks_no_thresh']} chunks):")
    print(f"  {row['answer_no_thresh'][:400]}")
    print(f"\n  THRESHOLD ≥ {THRESHOLD} ({row['chunks_with_thresh']} chunks):")
    print(f"  {row['answer_with_thresh'][:400]}")

---
## Documentation Questions

Fill in your observations after reviewing the outputs and charts above.

### 1. When is there a clear "winner" (large gap between #1 and #2)?
*Look at the `winner_gap` column and the score profile charts. Which query types tend to produce large gaps? What does a large gap tell you about the query–corpus match?*

---

### 2. When are scores tightly clustered (ambiguous retrieval)?
*Look at `score_std`. Which queries had the lowest spread? Were these off-topic queries, broad questions, or something else? What happens to answer quality when retrieval is ambiguous?*

---

### 3. What score threshold would you use to filter out irrelevant results?
*Look at the threshold sensitivity chart. At what value does the number of surviving queries start dropping sharply? Compare answer quality at your chosen threshold vs. no threshold for the off-topic query (Q09).*

---

### 4. How does score distribution correlate with answer quality?
*For queries where `winner_gap` is large, is the answer better? For queries where all scores are low (e.g. Q09 off-topic), does the model hallucinate or correctly say it doesn't know?*

In [ ]:
# Download all output files in Colab
output_files = [
    'exercise9_score_details.csv',
    'exercise9_score_summary.csv',
    'exercise9_score_profiles.png',
    'exercise9_gap_vs_spread.png',
    'exercise9_threshold_sensitivity.png',
]
try:
    from google.colab import files as colab_files
    for fname in output_files:
        if os.path.exists(fname):
            colab_files.download(fname)
            print(f' Downloading: {fname}')
        else:
            print(f' Not found (skipping): {fname}')
except ImportError:
    print('Files saved locally:')
    for fname in output_files:
        print(f'  {fname}')